In [1]:
import numpy as np
import pandas as pd
import emoji

from keras.models import Sequential
from keras.layers import Dense, LSTM, SimpleRNN, Embedding
from keras_preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical

In [2]:
data = pd.read_csv('data/emoji_data/emoji_data.csv', header=None)
data.head()

,0,1
0,French macaroon is so tasty,4
1,work is horrible,3
2,I am upset,3
3,throw the ball,1
4,Good joke,2


In [3]:
emoji_dictionary = {
    0: ":red_heart:",
    1: ":baseball:",
    2: ":grinning_face_with_big_eyes:",
    3: ":disappointed_face:",
    4: ":fork_and_knife_with_plate:"
}

def label_to_emoji(label):
    return emoji.emojize(emoji_dictionary[label])

In [4]:
X = data[0].values
Y = data[1].values

Y

array([4, 3, 3, 1, 2, 1, 4, 3, 4, 1, 3, 3, 2, 2, 4, 3, 2, 3, 3, 1, 3, 2,
       2, 2, 0, 1, 0, 4, 2, 0, 2, 0, 0, 3, 4, 0, 2, 1, 3, 1, 0, 4, 0, 3,
       0, 4, 2, 3, 4, 2, 2, 3, 0, 2, 2, 3, 2, 3, 2, 2, 3, 3, 0, 2, 3, 0,
       2, 0, 0, 2, 3, 2, 4, 1, 3, 3, 0, 0, 3, 2, 0, 3, 0, 2, 2, 4, 2, 2,
       0, 0, 2, 3, 0, 4, 2, 1, 2, 3, 3, 2, 3, 0, 3, 0, 2, 0, 2, 3, 4, 3,
       1, 3, 4, 3, 2, 3, 3, 3, 1, 4, 4, 2, 2, 1, 1, 2, 3, 2, 3, 4, 2, 3,
       0, 2, 0, 0, 4, 3, 4, 2, 3, 2, 3, 4, 2, 1, 2, 4, 3, 1, 3, 2, 3, 2,
       2, 3, 3, 2, 4, 0, 0, 0, 3, 0, 0, 1, 1, 2, 2, 2, 0, 3, 2, 3, 3, 1,
       2, 2, 4, 2, 3, 1, 2], dtype=int64)

Embeddings

In [5]:
with open('data/glove_dataset/glove.6B.100d.txt','r', encoding='utf8') as file:
    content = file.readlines()

In [6]:
embeddings = {}

for line in content:
    line = line.split()
    embeddings[line[0]] = np.array(line[1:], dtype=float)

convert input text into tokens

In [7]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)
word_to_index = tokenizer.word_index


In [8]:
Xtokens = tokenizer.texts_to_sequences(X)


In [9]:
def get_maxlen(data):
    maxlen = 0
    for sent in data:
        maxlen = max(maxlen, len(sent))
    
    return maxlen
maxlen = get_maxlen(Xtokens)

In [10]:
Xtrain = pad_sequences(Xtokens, maxlen = maxlen, padding = 'post', truncating = 'post')

In [11]:
Ytrain = to_categorical(Y)

Model

In [12]:
embed_size = 100

In [13]:
embedding_matrix = np.zeros((len(word_to_index)+1, embed_size))

for word, i in word_to_index.items():
    embed_vector = embeddings[word]
    embedding_matrix[i] = embed_vector

In [14]:
model = Sequential([
    Embedding(input_dim = len(word_to_index)+1,
              output_dim = embed_size,
              input_length=maxlen,
              weights = [embedding_matrix],
              trainable = True
              ),
    LSTM(units = 16, return_sequences=True),
    LSTM(units = 4),
    Dense(5, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [15]:
model.fit(Xtrain, Ytrain, epochs=1000)

Epoch 1/1000
6/6 [==============================] - 3s 8ms/step - loss: 1.6157 - accuracy: 0.1257
Epoch 2/1000
6/6 [==============================] - 0s 8ms/step - loss: 1.5897 - accuracy: 0.2732
Epoch 3/1000
6/6 [==============================] - 0s 9ms/step - loss: 1.5662 - accuracy: 0.3115
Epoch 4/1000
6/6 [==============================] - 0s 8ms/step - loss: 1.5460 - accuracy: 0.3224
Epoch 5/1000
6/6 [==============================] - 0s 15ms/step - loss: 1.5284 - accuracy: 0.3279
Epoch 6/1000
6/6 [==============================] - 0s 10ms/step - loss: 1.5095 - accuracy: 0.3388
Epoch 7/1000
6/6 [==============================] - 0s 9ms/step - loss: 1.4867 - accuracy: 0.3607
Epoch 8/1000
6/6 [==============================] - 0s 9ms/step - loss: 1.4574 - accuracy: 0.4481
Epoch 9/1000
6/6 [==============================] - 0s 9ms/step - loss: 1.4241 - accuracy: 0.5137
Epoch 10/1000
6/6 [==============================] - 0s 9ms/step - loss: 1.3835 - accuracy: 0.5410
Epoch 11/1000
6/6

KeyboardInterrupt: 

In [16]:
test = ["I am trying", "I want to cry", "This is just sad"]

test_seq = tokenizer.texts_to_sequences(test)
Xtest = pad_sequences(test_seq, maxlen=maxlen, padding='post', truncating='post')


y_pred = model.predict(Xtest)
y_pred = np.argmax(y_pred, axis=1)
y_pred

1/1 [==============================] - 1s 678ms/step


array([2, 2, 0], dtype=int64)

In [18]:
for i in range(len(test)):
    print(test[i], label_to_emoji(y_pred[i]))

I am trying 😃
I want to cry 😃
This is just sad ❤️
